## Import modules

In [1]:
import numpy as np

import pandas as pd
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 1600)

import re
import requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

#!conda install -c conda-forge folium=0.5.0 --yes
#import folium # map rendering library

!conda install -c conda-forge geopy --yes
import geopy.geocoders                     # convert an address into latitude and longitude values

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge


## Areas of London

In [2]:
areas_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_areas_of_London', header=0)[1]
areas_df.head()

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,Greenwich[1],LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[2]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[2],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[2],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


### Column names

In [3]:
areas_df.columns = areas_df.columns.str.replace('[^a-zA-Z]', '_')
areas_df.head()

,Location,London_borough,Post_town,Postcode_district,Dial_code,OS_grid_ref
0,Abbey Wood,Greenwich[1],LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[2]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[2],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[2],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


### Get rid of comments in London_borough column

In [4]:
areas_df['London_borough'].replace(to_replace='\[.*\]' ,value='', inplace=True, regex=True)
areas_df.head(10)

,Location,London_borough,Post_town,Postcode_district,Dial_code,OS_grid_ref
0,Abbey Wood,Greenwich,LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon,CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon,CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728
5,Aldborough Hatch,Redbridge,ILFORD,IG2,020,TQ455895
6,Aldgate,City,LONDON,EC3,020,TQ334813
7,Aldwych,Westminster,LONDON,WC2,020,TQ307810
8,Alperton,Brent,WEMBLEY,HA0,020,TQ185835
9,Anerley,Bromley,LONDON,SE20,020,TQ345695


Now let's note that some locations belong to more than one borough, e.g., `Acton` belongs to boroughs `Ealing` and `Hammersmith and Fulham`. We want to break down such locations to separate entries, so we need to split `London_borough`. Looking at the wiki page, we see that we have following potential separators: `,`, `&`, `and`. Yet, some borough names sontain `and` word, so let's look at entries with `,`, `&`, `and` in `London_borough` column closer.

In [5]:
areas_df[areas_df.London_borough.str.contains(',| and |&')]

,Location,London_borough,Post_town,Postcode_district,Dial_code,OS_grid_ref
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",020,TQ205805
19,Barking,Barking and Dagenham,BARKING,IG11,020,TQ440840
31,Becontree,Barking and Dagenham,DAGENHAM,RM9,020,TQ485855
32,Becontree Heath,Barking and Dagenham,DAGENHAM,RM8,020,TQ493871
68,Brompton,"Kensington and Chelsea,Hammersmith and Fulham",LONDON,SW3,020,TQ275795
83,Castle Green,Barking and Dagenham,DAGENHAM,RM9,020,TQ475837
85,Chadwell Heath,"Redbridge, Barking and Dagenham",ROMFORD,RM6,020,TQ485885
91,Chelsea,Kensington and Chelsea,LONDON,SW3,020,TQ275775
99,Chiswick,"Hounslow, Ealing, Hammersmith and Fulham",LONDON,W4,020,TQ205785
102,Clapham,"Lambeth, Wandsworth",LONDON,SW4,020,TQ295755


The above shows that all `and`s are part of borough names (`Hammersmith and Fulham`, `Barking and Dagenham`, `Kensington and Chelsea`), except these two cases:
* `Camden and Islington`
* `Haringey and Barnet`

Commas and ampersand (`Islington & City`), on the other hand, are indeed borough names separators, so let's do the split.

In [6]:
# first fix "Camden and Islington" and "Haringey and Barnet"
areas_df['London_borough'] = areas_df['London_borough'].str.replace('Camden and Islington', 'Camden,Islington')
areas_df['London_borough'] = areas_df['London_borough'].str.replace('Haringey and Barnet', 'Haringey,Barnet')

# then do the split
areas_split_df = pd.DataFrame(columns=['Location', 'London_borough'])
for index, row in areas_df.iterrows():
    location = row['Location']
    boroughs = re.sub(" *, *| +& +", ",", row['London_borough'])
    if "," in boroughs:
        for borough in boroughs.split(","):
            areas_split_df = areas_split_df.append({'Location': location + " (" + borough + ")", 'London_borough': borough}, ignore_index=True)
    else:
        areas_split_df = areas_split_df.append({'Location': location, 'London_borough': boroughs}, ignore_index=True)
areas_split_df.head(10)

,Location,London_borough
0,Abbey Wood,Greenwich
1,Acton (Ealing),Ealing
2,Acton (Hammersmith and Fulham),Hammersmith and Fulham
3,Addington,Croydon
4,Addiscombe,Croydon
5,Albany Park,Bexley
6,Aldborough Hatch,Redbridge
7,Aldgate,City
8,Aldwych,Westminster
9,Alperton,Brent


Do we have duplicate location names?

In [7]:
duplicates = areas_split_df['Location'].duplicated(keep=False) == True
areas_split_df[duplicates]

,Location,London_borough
38,Belmont,Harrow
39,Belmont,Sutton
105,Church End,Brent
106,Church End,Barnet
118,Coombe,Croydon
119,Coombe,Kingston upon Thames
208,Grove Park,Hounslow
209,Grove Park,Lewisham
242,Hayes,Bromley
243,Hayes,Hillingdon


Let's fix those duplicates.

In [8]:
for loc in areas_split_df[areas_split_df['Location'].duplicated(keep=False) == True]['Location'].index:
    areas_split_df.loc[loc, 'Location'] = areas_split_df.loc[loc, 'Location'] + ' (' + areas_split_df.loc[loc, 'London_borough'] + ')'
areas_split_df[duplicates]

,Location,London_borough
38,Belmont (Harrow),Harrow
39,Belmont (Sutton),Sutton
105,Church End (Brent),Brent
106,Church End (Barnet),Barnet
118,Coombe (Croydon),Croydon
119,Coombe (Kingston upon Thames),Kingston upon Thames
208,Grove Park (Hounslow),Hounslow
209,Grove Park (Lewisham),Lewisham
242,Hayes (Bromley),Bromley
243,Hayes (Hillingdon),Hillingdon


## London boroughs from Wikipedia

In [9]:
boroughs_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_London_boroughs', header=0, encoding = 'utf-8-sig')[0]
boroughs_df.head(10)

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ / 51.5607; 0.1557﻿ (Barking and Dagenham),25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ / 51.6252; -0.1517﻿ (Barnet),31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ / 51.4549; 0.1505﻿ (Bexley),23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ / 51.5588; -0.2817﻿ (Brent),12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ / 51.4039; 0.0198﻿ (Bromley),20
5,Camden,Y,NaN,Camden London Borough Council,Labour,"Camden Town Hall, Judd Street",8.40,229719,51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W﻿ / 51.5290; -0.1255﻿ (Camden),11
6,Croydon,NaN,NaN,Croydon London Borough Council,Labour,"Bernard Weatherill House, Mint Walk",33.41,372752,51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W﻿ / 51.3714; -0.0977﻿ (Croydon),19
7,Ealing,NaN,NaN,Ealing London Borough Council,Labour,"Perceval House, 14-16 Uxbridge Road",21.44,342494,51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W﻿ / 51.5130; -0.3089﻿ (Ealing),13
8,Enfield,NaN,NaN,Enfield London Borough Council,Labour,"Civic Centre, Silver Street",31.74,320524,51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W﻿ / 51.6538; -0.0799﻿ (Enfield),30
9,Greenwich [note 2],Y [note 3],Royal,Greenwich London Borough Council,Labour,"Woolwich Town Hall, Wellington Street",18.28,264008,51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E﻿ / 51.4892; 0.0648﻿ (Greenwich),22


Let's do some clean-up

In [10]:
# first remove notes from borough names
boroughs_df['Borough'].replace(to_replace='\[note.*$' ,value='', inplace=True, regex=True)
boroughs_df['Borough'] = boroughs_df['Borough'].str.strip()
boroughs_df.head(10)

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham,NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ / 51.5607; 0.1557﻿ (Barking and Dagenham),25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ / 51.6252; -0.1517﻿ (Barnet),31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ / 51.4549; 0.1505﻿ (Bexley),23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ / 51.5588; -0.2817﻿ (Brent),12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ / 51.4039; 0.0198﻿ (Bromley),20
5,Camden,Y,NaN,Camden London Borough Council,Labour,"Camden Town Hall, Judd Street",8.40,229719,51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W﻿ / 51.5290; -0.1255﻿ (Camden),11
6,Croydon,NaN,NaN,Croydon London Borough Council,Labour,"Bernard Weatherill House, Mint Walk",33.41,372752,51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W﻿ / 51.3714; -0.0977﻿ (Croydon),19
7,Ealing,NaN,NaN,Ealing London Borough Council,Labour,"Perceval House, 14-16 Uxbridge Road",21.44,342494,51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W﻿ / 51.5130; -0.3089﻿ (Ealing),13
8,Enfield,NaN,NaN,Enfield London Borough Council,Labour,"Civic Centre, Silver Street",31.74,320524,51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W﻿ / 51.6538; -0.0799﻿ (Enfield),30
9,Greenwich,Y [note 3],Royal,Greenwich London Borough Council,Labour,"Woolwich Town Hall, Wellington Street",18.28,264008,51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E﻿ / 51.4892; 0.0648﻿ (Greenwich),22


In [11]:
# next add population density
boroughs_df['Population density (ppl/sq. ml)'] = (boroughs_df['Population (2013 est)[1]'] / boroughs_df['Area (sq mi)']).round(2)
boroughs_df.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map,Population density (ppl/sq. ml)
0,Barking and Dagenham,NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ / 51.5607; 0.1557﻿ (Barking and Dagenham),25,13952.05
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ / 51.6252; -0.1517﻿ (Barnet),31,11020.84
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ / 51.4549; 0.1505﻿ (Bexley),23,10123.48
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ / 51.5588; -0.2817﻿ (Brent),12,18997.84
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ / 51.4039; 0.0198﻿ (Bromley),20,5483.85


In [12]:
# remove junk from coordinates
boroughs_df['Co-ordinates'].replace(to_replace=['^.*/ ', ' \(.*'] ,value=['', ''], inplace=True, regex=True)
boroughs_df.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map,Population density (ppl/sq. ml)
0,Barking and Dagenham,NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352,51.5607; 0.1557﻿,25,13952.05
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,369088,51.6252; -0.1517﻿,31,11020.84
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687,51.4549; 0.1505﻿,23,10123.48
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264,51.5588; -0.2817﻿,12,18997.84
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899,51.4039; 0.0198﻿,20,5483.85


In [13]:
# and split coordinates
boroughs_df[['Latitude', 'Longtitude']] = boroughs_df['Co-ordinates'].str.split('; ', expand=True)
# remove BOM and convert to float
boroughs_df['Latitude'] = boroughs_df['Latitude'].str.replace('\ufeff', '').astype(float)
boroughs_df['Longtitude'] = boroughs_df['Longtitude'].str.replace('\ufeff', '').astype(float)
boroughs_df.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map,Population density (ppl/sq. ml),Latitude,Longtitude
0,Barking and Dagenham,NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352,51.5607; 0.1557﻿,25,13952.05,51.5607,0.1557
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,369088,51.6252; -0.1517﻿,31,11020.84,51.6252,-0.1517
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687,51.4549; 0.1505﻿,23,10123.48,51.4549,0.1505
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264,51.5588; -0.2817﻿,12,18997.84,51.5588,-0.2817
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899,51.4039; 0.0198﻿,20,5483.85,51.4039,0.0198


In [14]:
# leave only needed columns
boroughs_df = boroughs_df[['Borough', 'Latitude', 'Longtitude', 'Population density (ppl/sq. ml)']]
boroughs_df.head()

,Borough,Latitude,Longtitude,Population density (ppl/sq. ml)
0,Barking and Dagenham,51.5607,0.1557,13952.05
1,Barnet,51.6252,-0.1517,11020.84
2,Bexley,51.4549,0.1505,10123.48
3,Brent,51.5588,-0.2817,18997.84
4,Bromley,51.4039,0.0198,5483.85


## Check validity of boroughs in areas dataframe

In [15]:
boroughs = boroughs_df['Borough'].tolist()
boroughs.append('City') # The City of London is the 33rd principal division of Greater London but it is not a London borough.
for index, row in areas_split_df.iterrows():
    borough = row['London_borough']
    if not borough in boroughs:
        print('Warning: "%s" is invalid borough' % borough)

`Dartford` formally belongs to Kent, not London. Let's drop invalid entry.

In [16]:
areas_split_df = areas_split_df[areas_split_df['London_borough'] != "Dartford"]

## GFR (General fertility rate) dataset

In [17]:
gfr_df = pd.read_csv('https://raw.githubusercontent.com/alllexx88/Coursera_Capstone/94e04598753242c7970d559ed4db92889100ae95/london-births-fertility-rates-borough-2017.csv')
print('dataset shape:', gfr_df.shape)
gfr_df.head()

dataset shape: (32, 2)


,Borough,GFR
0,Barking and Dagenham,82.6
1,Barnet,64.3
2,Bexley,63.3
3,Brent,74.5
4,Bromley,66.7


Let's check validity of borough names, like we did for areas dataset.

In [18]:
for index, row in gfr_df.iterrows():
    borough = row['Borough']
    if not borough in boroughs:
        print('Warning: "%s" is invalid borough' % borough)

Looks like `Hackney` and `City of Londond` are counted as one in the birth-rate dataset, hence let's rename `Hackney` and `City` boroughs to `Hackney and City of London` in the areas dataset. While we're at it, let's rename column `London_borough` to `Borough`, for future joining.

In [19]:
areas_split_df.rename(index=str, columns={'London_borough': 'Borough'}, inplace=True)
areas_split_df['Borough'] = areas_split_df['Borough'].str.replace('City|Hackney', 'Hackney and City of London')
areas_split_df[areas_split_df['Borough'] == 'Hackney and City of London']

,Location,Borough
7,Aldgate,Hackney and City of London
19,Barbican,Hackney and City of London
50,Blackfriars,Hackney and City of London
141,Dalston,Hackney and City of London
179,Farringdon (City),Hackney and City of London
211,Hackney,Hackney and City of London
212,Hackney Central,Hackney and City of London
213,Hackney Marshes,Hackney and City of London
214,Hackney Wick,Hackney and City of London
216,Haggerston,Hackney and City of London


## Areas coordinates using geopy

In [20]:
def retrieve_geocode_london(location_string):
    import geopy.geocoders
    geopy.geocoders.options.default_user_agent = "my-application"
    geolocator = geopy.geocoders.Yandex()
    location = None
    address = location_string + ', London, UK'
    while location is None:
        location = geolocator.geocode(address)
    return location.latitude, location.longitude

In [21]:
# this csv was generated by repeatedly calling the cycle below: load it for faster execution
areas_coord_df = pd.read_csv('https://raw.githubusercontent.com/alllexx88/Coursera_Capstone/d29b5662742e07d974da60abd3afbfd0f0c62ff1/london-locations-coord.csv')

# join with main areas dataframe
areas_split_df = pd.merge(areas_split_df, areas_coord_df, how='left', on='Location')

df_len = areas_split_df.shape[0]
for i, j in enumerate(areas_split_df.iterrows()):
    index = j[0]
    row = j[1]
    if (not 'Latitude' in areas_split_df.columns) or np.isnan(areas_split_df.loc[index, 'Latitude']):
        latitude, longtitude = retrieve_geocode_london(row['Location'])
        areas_split_df.loc[index, 'Latitude'] = latitude
        areas_split_df.loc[index, 'Longtitude'] = longtitude
    if (i + 1) % 10 == 0:
        print('{}/{} completed - {}'.format(i + 1, df_len, row['Location']))

10/560 completed - Alperton
20/560 completed - Barbican
30/560 completed - Bayswater
40/560 completed - Belmont (Sutton)
50/560 completed - Blackfen
60/560 completed - Bowes Park
70/560 completed - Brompton (Kensington and Chelsea)
80/560 completed - Canary Wharf
90/560 completed - Chalk Farm
100/560 completed - Chinbrook
110/560 completed - Clerkenwell
120/560 completed - Coombe (Kingston upon Thames)
130/560 completed - Cricklewood (Brent)
140/560 completed - Custom House
150/560 completed - Dulwich
160/560 completed - East Wickham
170/560 completed - Eltham
180/560 completed - Feltham
190/560 completed - Fortis Green
200/560 completed - Goddington
210/560 completed - Gunnersbury
220/560 completed - Hammersmith
230/560 completed - Harlington
240/560 completed - Hatton
250/560 completed - Highgate
260/560 completed - Horn Park
270/560 completed - Kenley
280/560 completed - Kidbrooke
290/560 completed - Lambeth
300/560 completed - Leytonstone
310/560 completed - Loxford
320/560 complet

In [22]:
areas_split_df.tail(10)

,Location,Borough,Latitude,Longtitude
550,Woodford Green,Redbridge,51.602595,0.046191
551,Woodlands,Hounslow,51.464699,-0.328882
552,Woodside,Croydon,51.387497,-0.065784
553,Woodside Park,Barnet,51.618026,-0.185071
554,Woolwich,Greenwich,51.487138,0.063394
555,Worcester Park (Sutton),Sutton,51.382697,-0.243488
556,Worcester Park (Kingston upon Thames),Kingston upon Thames,51.382697,-0.243488
557,Wormwood Scrubs,Hammersmith and Fulham,51.507351,-0.127660
558,Yeading,Hillingdon,51.521565,-0.393184
559,Yiewsley,Hillingdon,51.515496,-0.472882


## POIs (places of interest) for each London location

In [23]:
# The code was removed by Watson Studio for sharing.

In [24]:
VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [25]:
def getNearbyVenueCategories(names, latitudes, longtitudes, radius=500):
    count_len = names.shape[0]
    venues_list=[]
    for i, j in enumerate(zip(names, latitudes, longtitudes)):
        name = j[0]
        lat  = j[1]
        lng  = j[2]
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
                    # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                re.search('categories_v2/(.*)/', v['venue']['categories'][0]['icon']['prefix'], re.IGNORECASE).group(1)) for v in results])              
        except:
            venues_list.append([(
                name, 
                lat, 
                lng, 
                np.nan, 
                np.nan, 
                np.nan,  
                np.nan)])
        if (i + 1) % 50 == 0:
            print('{} of {} completed'.format(i + 1, count_len))
                
        


    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
venues_df = getNearbyVenueCategories(names=areas_split_df['Location'],
                                     latitudes=areas_split_df['Latitude'],
                                     longtitudes=areas_split_df['Longtitude'],
                                     radius=1000
                                    )
print('Shape: {}'.format(venues_df.shape))
venues_df.head(10)

50 of 560 completed
100 of 560 completed
150 of 560 completed
200 of 560 completed
250 of 560 completed
300 of 560 completed
350 of 560 completed
400 of 560 completed
450 of 560 completed
500 of 560 completed
550 of 560 completed
Shape: (27494, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abbey Wood,51.487621,0.11405,Sainsbury's,51.492824,0.120724,shops
1,Abbey Wood,51.487621,0.11405,Lesnes Abbey,51.489526,0.125839,arts_entertainment
2,Abbey Wood,51.487621,0.11405,Co-op Food,51.487650,0.113490,shops
3,Abbey Wood,51.487621,0.11405,Co-op Food,51.494124,0.112796,shops
4,Abbey Wood,51.487621,0.11405,Abbey Wood Railway Station (ABW),51.491049,0.119893,travel
5,Abbey Wood,51.487621,0.11405,Abbey Wood Caravan Club,51.485502,0.120014,parks_outdoors
6,Abbey Wood,51.487621,0.11405,Bean @ Work,51.491172,0.120649,food
7,Abbey Wood,51.487621,0.11405,Costcutter,51.491287,0.120938,shops
8,Abbey Wood,51.487621,0.11405,Bostall Woods,51.480026,0.115305,parks_outdoors
9,Abbey Wood,51.487621,0.11405,Meghna Tandoori,51.485709,0.101681,food


One-hot encode venues dataframe.

In [27]:
venues_onehot_df = pd.get_dummies(venues_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_onehot_df['Neighborhood'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
venues_onehot_df = venues_onehot_df[[venues_onehot_df.columns[-1]] + list(venues_onehot_df.columns[:-1])]

venues_onehot_df.head()

,Neighborhood,arts_entertainment,building,education,event,food,nightlife,parks_outdoors,shops,travel
0,Abbey Wood,0,0,0,0,0,0,0,1,0
1,Abbey Wood,1,0,0,0,0,0,0,0,0
2,Abbey Wood,0,0,0,0,0,0,0,1,0
3,Abbey Wood,0,0,0,0,0,0,0,1,0
4,Abbey Wood,0,0,0,0,0,0,0,0,1


Group venues by neighborhood and sum over categories.

In [28]:
venues_grouped_df = venues_onehot_df.groupby('Neighborhood').sum().reset_index()
venues_grouped_df.head()

,Neighborhood,arts_entertainment,building,education,event,food,nightlife,parks_outdoors,shops,travel
0,Abbey Wood,1,0,0,0,2,0,2,4,1
1,Acton (Ealing),1,3,0,0,23,8,3,12,7
2,Acton (Hammersmith and Fulham),2,0,0,0,6,1,1,5,3
3,Addington,0,0,0,0,3,0,1,2,1
4,Addiscombe,0,0,0,0,7,2,5,6,3


## Merge areas and venues dataframes and group by boroughs

In [29]:
joint_boroughs_df = pd.merge(areas_split_df, venues_grouped_df, left_on = 'Location', right_on = 'Neighborhood', how='inner')
joint_boroughs_df.drop(['Neighborhood', 'Location', 'Latitude', 'Longtitude'], axis=1, inplace=True)
joint_boroughs_df = joint_boroughs_df.groupby('Borough').sum().reset_index()
joint_boroughs_df

,Borough,arts_entertainment,building,education,event,food,nightlife,parks_outdoors,shops,travel
0,Barking and Dagenham,12.0,10.0,0.0,0.0,79.0,24.0,22.0,60.0,16.0
1,Barnet,123.0,59.0,2.0,0.0,664.0,82.0,113.0,270.0,107.0
2,Bexley,161.0,32.0,0.0,0.0,479.0,106.0,110.0,235.0,90.0
3,Brent,34.0,25.0,1.0,0.0,375.0,54.0,55.0,186.0,81.0
4,Bromley,78.0,44.0,0.0,1.0,504.0,135.0,85.0,218.0,81.0
5,Camden,197.0,46.0,0.0,0.0,860.0,198.0,107.0,252.0,101.0
6,Croydon,47.0,8.0,0.0,0.0,197.0,38.0,60.0,106.0,51.0
7,Ealing,42.0,23.0,1.0,1.0,268.0,53.0,47.0,88.0,43.0
8,Enfield,33.0,13.0,0.0,0.0,189.0,43.0,42.0,110.0,40.0
9,Greenwich,77.0,16.0,1.0,0.0,240.0,76.0,68.0,143.0,63.0


Add in population density

In [30]:
joint_boroughs_df['population_density'] = boroughs_df['Population density (ppl/sq. ml)']
joint_boroughs_df

,Borough,arts_entertainment,building,education,event,food,nightlife,parks_outdoors,shops,travel,population_density
0,Barking and Dagenham,12.0,10.0,0.0,0.0,79.0,24.0,22.0,60.0,16.0,13952.05
1,Barnet,123.0,59.0,2.0,0.0,664.0,82.0,113.0,270.0,107.0,11020.84
2,Bexley,161.0,32.0,0.0,0.0,479.0,106.0,110.0,235.0,90.0,10123.48
3,Brent,34.0,25.0,1.0,0.0,375.0,54.0,55.0,186.0,81.0,18997.84
4,Bromley,78.0,44.0,0.0,1.0,504.0,135.0,85.0,218.0,81.0,5483.85
5,Camden,197.0,46.0,0.0,0.0,860.0,198.0,107.0,252.0,101.0,27347.50
6,Croydon,47.0,8.0,0.0,0.0,197.0,38.0,60.0,106.0,51.0,11156.90
7,Ealing,42.0,23.0,1.0,1.0,268.0,53.0,47.0,88.0,43.0,15974.53
8,Enfield,33.0,13.0,0.0,0.0,189.0,43.0,42.0,110.0,40.0,10098.42
9,Greenwich,77.0,16.0,1.0,0.0,240.0,76.0,68.0,143.0,63.0,14442.45


## Merge with GFR dataframe

In [31]:
joint_boroughs_df = pd.merge(joint_boroughs_df, gfr_df, how='inner', on='Borough')
joint_boroughs_df

,Borough,arts_entertainment,building,education,event,food,nightlife,parks_outdoors,shops,travel,population_density,GFR
0,Barking and Dagenham,12.0,10.0,0.0,0.0,79.0,24.0,22.0,60.0,16.0,13952.05,82.6
1,Barnet,123.0,59.0,2.0,0.0,664.0,82.0,113.0,270.0,107.0,11020.84,64.3
2,Bexley,161.0,32.0,0.0,0.0,479.0,106.0,110.0,235.0,90.0,10123.48,63.3
3,Brent,34.0,25.0,1.0,0.0,375.0,54.0,55.0,186.0,81.0,18997.84,74.5
4,Bromley,78.0,44.0,0.0,1.0,504.0,135.0,85.0,218.0,81.0,5483.85,66.7
5,Camden,197.0,46.0,0.0,0.0,860.0,198.0,107.0,252.0,101.0,27347.50,41.5
6,Croydon,47.0,8.0,0.0,0.0,197.0,38.0,60.0,106.0,51.0,11156.90,72.5
7,Ealing,42.0,23.0,1.0,1.0,268.0,53.0,47.0,88.0,43.0,15974.53,71.0
8,Enfield,33.0,13.0,0.0,0.0,189.0,43.0,42.0,110.0,40.0,10098.42,68.3
9,Greenwich,77.0,16.0,1.0,0.0,240.0,76.0,68.0,143.0,63.0,14442.45,69.1


## Regression models

In [32]:
joint_boroughs_X = joint_boroughs_df[joint_boroughs_df.columns[1:-1]]
joint_boroughs_Y = joint_boroughs_df['GFR']

Import modules

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score as R2
from sklearn.svm import SVR

Train/test split

In [34]:
X_train, X_test, y_train, y_test = train_test_split(joint_boroughs_X, joint_boroughs_Y, test_size=0.2, random_state=1)

Linear and SVR pipelines

In [35]:
pipe_lr = Pipeline([
        ('scale', StandardScaler()),
        ('pca', PCA(n_components=5)),
        ('regr', Ridge(alpha=50))])

pipe_sv = Pipeline([
        ('scale', StandardScaler()),
        ('pca', PCA(n_components=5)),
        ('regr', SVR(C=1000, gamma=0.1, epsilon=0.1))])

### Grid search Ridge pipeline

In [36]:
parameters_lr = dict(regr__alpha = np.linspace(1, 100))

grid_lr = GridSearchCV (pipe_lr, parameters_lr, cv=5, scoring='r2', return_train_score=False, verbose=1, n_jobs=-1)

grid_lr.fit(X_train, y_train)

print("Best alpha parameter found on training set:")
print()
print(grid_lr.best_params_['regr__alpha'])
cv_results = pd.DataFrame(grid_lr.cv_results_).sort_values(by=['mean_test_score'], ascending=False)
cv_results.drop(list(cv_results.filter(regex = 'params|time|rank_test_score')), axis = 1, inplace = True)
cv_results.head(8)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   16.3s finished


Best alpha parameter found on training set:

41.4081632653


,mean_test_score,param_regr__alpha,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,std_test_score
20,-0.474053,41.4082,-0.408544,-0.122435,-2.706054,0.534788,0.331980,1.164234
19,-0.474162,39.3878,-0.458497,-0.115182,-2.674662,0.539295,0.338235,1.153984
21,-0.474362,43.4286,-0.361860,-0.129256,-2.736906,0.530349,0.325863,1.174790
18,-0.474734,37.3673,-0.512045,-0.107443,-2.642690,0.543869,0.344636,1.144137
22,-0.475051,45.449,-0.318161,-0.135692,-2.767253,0.525975,0.319878,1.185575
17,-0.475819,35.3469,-0.569558,-0.099158,-2.610082,0.548512,0.351192,1.134808
23,-0.476083,47.4694,-0.277194,-0.141781,-2.797125,0.521666,0.314019,1.196521
24,-0.477428,49.4898,-0.238735,-0.147557,-2.826545,0.517419,0.308280,1.207576


Let's check R2 score and RMSE on the test set.

In [37]:
y_lr_pred = grid_lr.predict(X_test)

print('Linear regression RMSE: {} live births per 1,000 women aged 15-44'.format(np.sqrt(MSE(y_test, y_lr_pred)).round(2)))
print('Linear regression R2 score: {}'.format(np.sqrt(R2(y_test, y_lr_pred)).round(2)))

Linear regression RMSE: 7.73 live births per 1,000 women aged 15-44
Linear regression R2 score: 0.66


London has GFR of 62.9 ([see here](https://data.london.gov.uk/dataset/births-and-fertility-rates-borough)), hence RMSE of 7.73 is approximately 12.29% error.

### SVR randomized parameters search

In [38]:
parameters_sv = dict(regr__C = np.linspace(0.1, 1000, 10000),
                     regr__gamma = np.linspace(0.1,1000,10000),
                     regr__epsilon = np.linspace(0.0001,1,10000))

rand_sv = RandomizedSearchCV(pipe_sv, param_distributions=parameters_sv, cv=5, random_state=1,
                             scoring='r2', return_train_score=False,  n_iter=10000, verbose=1, n_jobs=-1)
rand_sv.fit(X_train, y_train)

print("Best parameters found on training set:")
print()
print('C={}, gamma={}, epsilon={}'.format(rand_sv.best_params_['regr__C'],
                                          rand_sv.best_params_['regr__gamma'],
                                          rand_sv.best_params_['regr__epsilon']))

cv_results = pd.DataFrame(rand_sv.cv_results_).sort_values(by=['mean_test_score'], ascending=False)
cv_results.drop(list(cv_results.filter(regex = 'params|time|rank_test_score')), axis = 1, inplace = True)
cv_results.head(8)

Fitting 5 folds for each of 10000 candidates, totalling 50000 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 1184 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 5984 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done 9584 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done 13984 tasks      | elapsed:   42.1s
[Parallel(n_jobs=-1)]: Done 19184 tasks      | elapsed:   57.2s
[Parallel(n_jobs=-1)]: Done 25184 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 31984 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 39584 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 47984 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 50000 out of 50000 | elapsed:  2.4min finished


Best parameters found on training set:

C=160.29999999999998, gamma=0.1, epsilon=0.8863


,mean_test_score,param_regr__C,param_regr__epsilon,param_regr__gamma,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,std_test_score
8180,-0.013880,160.3,0.8863,0.1,-1.632797,0.606946,-0.158797,0.755843,0.359405,0.867079
6371,-0.195386,99.6,0.2272,0.4,-0.630284,0.229958,-1.235050,0.541160,0.117286,0.646718
1948,-0.336894,700.6,0.6474,0.1,-1.355771,-1.837411,0.487689,0.655984,0.365038,1.043850
808,-0.432153,241.2,0.5744,0.3,-0.522285,-0.007914,-2.005398,0.448654,-0.073824,0.844680
9173,-0.507837,897.6,0.3316,0.1,-1.371529,-2.675681,0.500812,0.645104,0.362111,1.307591
7856,-0.726514,32.4,0.0345,0.9,-0.887116,0.323350,-3.599231,0.496770,0.033658,1.513591
9914,-1.078441,591.4,0.0864,5.6,-0.280626,-0.319062,-4.733712,0.104847,-0.163653,1.833640
1518,-1.078473,998.5,0.2024,5.5,-0.286367,-0.313136,-4.740024,0.106872,-0.159711,1.836807


R2 score and RMSE on the test set.

In [39]:
y_sv_pred = rand_sv.predict(X_test)

print('SVR RMSE: {} live births per 1,000 women aged 15-44'.format(np.sqrt(MSE(y_test, y_sv_pred)).round(2)))
print('SVR R2 score: {}'.format(np.sqrt(R2(y_test, y_sv_pred)).round(2)))

SVR RMSE: 6.19 live births per 1,000 women aged 15-44
SVR R2 score: 0.8


London has GFR of 62.9 ([see here](https://data.london.gov.uk/dataset/births-and-fertility-rates-borough)), hence RMSE of 6.19 is approximately 9.84% error.